Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [23]:

import logging
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from icecream import ic

Cost: $d + (d \cdot \alpha \cdot w)^\beta$ with $\alpha \ge 0$ and $\beta \ge 0$

## Genetico

In [ ]:
from src.GA_solver import GA_Solver

In [101]:
from icecream import List, Tuple
from Problem import Problem

def check_feasibility_without_start_depot(
    problem: Problem,
    solution: List[Tuple[int, float]],
) -> bool:
    """
    Checks if a solution is feasible:
    1. Each step must be between adjacent cities
    2. All gold from all cities must be collected (at least once)
    
    :param problem: Problem instance
    :param solution: List of (city, gold_picked)
    :return: True if feasible, False otherwise
    """
    graph = problem.graph
    gold_at = nx.get_node_attributes(graph, "gold")
    
    # Track collected gold per city
    gold_collected = {}
    prev_city = 0  # Start from depot
    
    current_weight = 0
    i=0
    
    for city, gold in solution:
        # Check adjacency
        if not graph.has_edge(prev_city, city):
            print(f"❌ Feasibility failed: no edge between {prev_city} and {city} i={i}")
            print(f"Path segment: {prev_city} -> {city}")
            print( solution)
            return False
        
        # Track collected gold
        if gold > 0:
            gold_collected[city] = gold_collected.get(city, 0.0) + gold
        
        # Update current weight
        current_weight += gold
        if city == 0:
            current_weight = 0
            
        prev_city = city
    
    # Verify all gold was collected
    for city in graph.nodes():
        if city == 0:  # Depot has no gold
            continue
        expected_gold = gold_at.get(city, 0.0)
        collected_gold = gold_collected.get(city, 0.0)
        
        if abs(expected_gold - collected_gold) > 1e-4:  # Float tolerance
            print(f"❌ Feasibility failed: city {city} i={i} has {expected_gold:.2f} gold, collected {collected_gold:.2f}")
            return False
        i += 1
    
    return True

#check_feasibility(p, best_path)

In [ ]:
from itertools import product
from Problem import Problem
import importlib
from Problem import Problem
from time import time
#import s345905 as solution_module
import src.utils as utils_module
import src.GA_solver as GA_solver_module
from src.utils import compute_ga_params
from src.utils import check_path

#importlib.reload(solution_module)
importlib.reload(utils_module)
importlib.reload(GA_solver_module)

GA_Solver= GA_solver_module.GA_Solver
check_path = utils_module.check_path
compute_ga_params = utils_module.compute_ga_params
#s345905_solution= solution_module.solution

cities_list = [1000 ]
densities = [0.2, 0.8]
alphas = [1]
betas = [0.1, 1, 2]

results = []

for n_cities, density, alpha, beta in product(cities_list, densities, alphas, betas):
    
    p = Problem(n_cities, density=density, alpha=alpha, beta=beta)
    t= time()
    pop, gen, off= compute_ga_params(n_cities=p.graph.number_of_nodes(), beta=p.beta, alpha=p.alpha)
    solver= GA_Solver(p, pop, gen, off)
    print("Initializing GA solver...")
    best_path , best_cost = solver.solution(fast=False)
    end_t= time()
    baseline_cost = p.baseline()
    is_valid = solver.check_feasibility(best_path)
    elapsed_time = end_t - t
    print(
        f"PROBLEM: {n_cities:4d} | d={density:.1f} | "
        f"a={alpha:.1f} | b={beta:.1f} | "
        f"GA PARAMETERS pop={pop:4d} | gen={gen:4d} | off={off:4d} | "
        f"baseline={baseline_cost:.2f} | ga={best_cost:.2f} | "
        f"time={elapsed_time:.2f}s | minutes={elapsed_time/60:.2f}m   |"
        f"rel={(best_cost/ baseline_cost if baseline_cost > 0 else 0) * 100:.2f}% |"
        f"is valid: {is_valid}"
    ) 
    results.append({
        "n_cities": n_cities,
        "density": density,
        "alpha": alpha,
        "beta": beta,
        "baseline_cost": baseline_cost,
        "ga_cost": best_cost,
        "relative_cost": best_cost /baseline_cost * 100
    })


Initializing GA solver...
PROBLEM: 1000 | d=0.2 | a=1.0 | b=0.1 | GA PARAMETERS pop=  30 | gen=  39 | off=   9 | baseline=4040.62 | ga=3309.50 | time=142.91s | minutes=2.38m   |rel=81.91% |is valid: True
Initializing GA solver...
PROBLEM: 1000 | d=0.2 | a=1.0 | b=1.0 | GA PARAMETERS pop=  30 | gen=  35 | off=   9 | baseline=195402.96 | ga=195358.29 | time=96.16s | minutes=1.60m   |rel=99.98% |is valid: True
Initializing GA solver...
PROBLEM: 1000 | d=0.2 | a=1.0 | b=2.0 | GA PARAMETERS pop=   3 | gen=  10 | off=   2 | baseline=37545927.70 | ga=2169683.50 | time=89.04s | minutes=1.48m   |rel=5.78% |is valid: True
Initializing GA solver...
PROBLEM: 1000 | d=0.8 | a=1.0 | b=0.1 | GA PARAMETERS pop=  30 | gen=  39 | off=   9 | baseline=2661.53 | ga=2365.35 | time=1129.57s | minutes=18.83m   |rel=88.87% |is valid: True
Initializing GA solver...
PROBLEM: 1000 | d=0.8 | a=1.0 | b=1.0 | GA PARAMETERS pop=  30 | gen=  35 | off=   9 | baseline=192953.57 | ga=192823.89 | time=730.12s | minutes=12

In [ ]:
from Problem import Problem

importlib.reload(GA_solver_module)
importlib.reload(utils_module)

GA_Solver= GA_solver_module.GA_Solver
compute_ga_params = utils_module.compute_ga_params
print("Testing baseline and improved baseline feasibility...")
p = Problem(num_cities=1000, density=0.8, alpha=0.2, beta=0.1)
    
pop, gen, off= compute_ga_params(n_cities=p.graph.number_of_nodes(), beta=p.beta, alpha=p.alpha)
solver= GA_Solver(p, pop, gen, off)
print("Ga_Solver initialized.   ")

u_path, u_cost=solver.solution(fast=False)
is_valid= solver.check_feasibility(u_path)

print(
        f"PROBLEM: {n_cities:4d} | d={density:.1f} | "
        f"a={alpha:.1f} | b={beta:.1f} | "
        f"GA PARAMETERS pop={pop:4d} | gen={gen:4d} | off={off:4d} | "
        f"baseline={baseline_cost:.2f} | ga={best_cost:.2f} | "
        f"time={elapsed_time:.2f}s | minutes={elapsed_time/60:.2f}m   |"
        f"rel={(best_cost/ baseline_cost if baseline_cost > 0 else 0) * 100:.2f}% |"
        f"is valid: {is_valid}"
    )

Testing baseline and improved baseline feasibility...
Ga_Solver initialized.   
PROBLEM: 1000 | d=0.8 | a=0.2 | b=0.1 | GA PARAMETERS pop=  30 | gen=  39 | off=   9 | baseline=3754515827.91 | ga=192727289.83 | time=158.74s | minutes=2.65m   |rel=5.13% |is valid: True


In [2]:
import os
import psutil

# Core totali disponibili sulla macchina
total_cores = os.cpu_count()
print(f"Core totali: {total_cores}")

# Core effettivamente utilizzabili dal processo corrente
# (su alcuni sistemi può differire dal totale)
p = psutil.Process()
print(f"Core utilizzati dal processo: {len(p.cpu_affinity()) if hasattr(p, 'cpu_affinity') else 'N/A'}")

Core totali: 8
Core utilizzati dal processo: 8


In [19]:
from Problem import Problem
import importlib
from Problem import Problem
from time import time
#import s345905 as solution_module
import src.utils as utils_module
import src.GA_solver as GA_solver_module
from src.utils import compute_ga_params
from src.utils import check_path

#importlib.reload(solution_module)
importlib.reload(utils_module)
importlib.reload(GA_solver_module)

GA_Solver= GA_solver_module.GA_Solver
check_path = utils_module.check_path


p= Problem(50, density=0.2, alpha=0.2, beta=2)
print(f"--- PROBLEM: {p.graph.number_of_nodes()} cities, alpha {p.alpha}, beta {p.beta} ---")
print(f"Baseline Cost: {p.baseline()}")

t= time()
pop, gen, off= compute_ga_params(n_cities=p.graph.number_of_nodes(), beta=p.beta, alpha=p.alpha)
solver= GA_Solver(p, pop, gen, off)
best_path , best_cost = solver.solution(fast=False)
is_valid, cost =check_path(p, best_path)
print(f"GA Solution valid: {is_valid} | GA Cost: {cost:.2f}")
end_time = time()
elapsed_time = end_time - t
baseline_cost = p.baseline()
print(f"GA Cost: {best_cost} Baseline Cost: {baseline_cost} | Relative cost: {(best_cost/baseline_cost)*100:.2f}% | Time taken: {elapsed_time/60:.2f} minutes\n")


--- PROBLEM: 50 cities, alpha 0.2, beta 2 ---
Baseline Cost: 105000.84011982652
GA Params: pop_size=50, generations=100, offprint=30
GA Solution valid: True | GA Cost: 5591.75
GA Cost: 5591.7521066109175 Baseline Cost: 105000.84011982652 | Relative cost: 5.33% | Time taken: 0.59 minutes



In [102]:
# TESTING SOLUTION MODULE
from Problem import Problem
import importlib
from Problem import Problem
from time import time
import s345905 as solution_module

importlib.reload(solution_module)

solution= solution_module.solution


p= Problem(50, density=0.2, alpha=0.2, beta=2)
print(f"--- PROBLEM: {p.graph.number_of_nodes()} cities, alpha {p.alpha}, beta {p.beta} ---")
print(f"Baseline Cost: {p.baseline()}")

t= time()
best_path= solution(p)
is_valid =check_feasibility_without_start_depot(p, best_path)
print(f"GA Solution valid: {is_valid} ")
end_time = time()
elapsed_time = end_time - t
baseline_cost = p.baseline()
print(f"Time taken: {elapsed_time:.2f} seconds\n")


--- PROBLEM: 50 cities, alpha 0.2, beta 2 ---
Baseline Cost: 105000.84011982652
GA Solution valid: True 
Time taken: 36.54 seconds



In [98]:
from Problem import Problem
import importlib
from Problem import Problem
from time import time
#import s345905 as solution_module
import src.utils as utils_module
import src.GA_solver as GA_solver_module
from src.utils import compute_ga_params
from src.utils import check_path

#importlib.reload(solution_module)
importlib.reload(utils_module)
importlib.reload(GA_solver_module)

GA_Solver= GA_solver_module.GA_Solver
check_path = utils_module.check_path

logging.getLogger().setLevel(logging.WARNING)

test_cases = [
    # 1. Il caso "Standard" (Equilibrio)
    Problem(50, density=0.4, alpha=1.0, beta=2),
    
    # 2. Il "Corriere Espresso" (Peso quasi irrilevante, vince chi fa meno km)
    Problem(100, density=0.7, alpha=0.05, beta=0.1),
    
    # 3. Il "Trasporto Eccezionale" (Peso punitivo, vince chi torna spesso al deposito)
    Problem(200, density=0.5, alpha=2, beta=0.9),
    
    # 4. L'incubo non-lineare (Costo esplosivo col peso)
    Problem(500, density=0.3, alpha=0.8, beta=2),
    
    # 5. Grafo Labirintico (Pochi collegamenti, obbliga a percorsi tortuosi)
    Problem(700, density=0.05, alpha=0.5, beta=1),
    
    # 6. Il caso "Super-Alpha" (Quasi ogni città richiede un viaggio dedicato)
    Problem (1000, density=1, alpha=1, beta=1),
]

for p in test_cases:
    print(f"--- PROBLEM: {p.graph.number_of_nodes()} cities, alpha {p.alpha}, beta {p.beta} ---")
    print(f"Baseline Cost: {p.baseline()}")

    t= time()
    pop, gen, off= compute_ga_params(n_cities=p.graph.number_of_nodes(), beta=p.beta, alpha=p.alpha)
    solver= GA_Solver(p, pop, gen, off)
    best_path , best_cost = solver.solution(fast=False)
    end_time = time()
    is_valid =solver.check_feasibility(best_path)
    
    elapsed_time = end_time - t
    print(f"GA Cost: {best_cost} | Relative cost: {best_cost/p.baseline()*100:.2f}% | Time taken: {elapsed_time/60:.2f} minutes, {elapsed_time%60:.2f} seconds\n  | Valid: {is_valid}\n")

--- PROBLEM: 50 cities, alpha 1.0, beta 2 ---
Baseline Cost: 1944918.0867871945
GA Cost: 78273.29064659135 | Relative cost: 4.02% | Time taken: 0.62 minutes, 37.38 seconds
  | Valid: True

--- PROBLEM: 100 cities, alpha 0.05, beta 0.1 ---
Baseline Cost: 222.18024514541358
GA Cost: 176.28043171627257 | Relative cost: 79.34% | Time taken: 0.33 minutes, 19.55 seconds
  | Valid: True

--- PROBLEM: 200 cities, alpha 2, beta 0.9 ---
Baseline Cost: 41475.39480993754
GA Cost: 40195.54838775443 | Relative cost: 96.91% | Time taken: 0.22 minutes, 13.19 seconds
  | Valid: True

--- PROBLEM: 500 cities, alpha 0.8, beta 2 ---
Baseline Cost: 12920597.006418131
GA Cost: 702092.950861605 | Relative cost: 5.43% | Time taken: 0.33 minutes, 19.58 seconds
  | Valid: True

--- PROBLEM: 700 cities, alpha 0.5, beta 1 ---
Baseline Cost: 86073.71668163202
GA Cost: 86051.34284561884 | Relative cost: 99.97% | Time taken: 0.24 minutes, 14.50 seconds
  | Valid: True

--- PROBLEM: 1000 cities, alpha 1, beta 1 ---
B

In [ ]:
from Problem import Problem 
import s345905 as solution_module
importlib.reload(solution_module)
s345905_solution= solution_module.solution

p= Problem(50, density=0.4, alpha=1.0, beta=1.0)
best_path=s345905_solution(p)
print(f"\n\nGA Path: {best_path}")



GA Path: [(0, 0), (46, 267.7086874288017), (0, 0), (46, 0), (23, 85.3598768187492), (46, 0), (0, 0), (46, 0), (23, 0), (16, 856.7576698082831), (23, 0), (46, 0), (0, 0), (44, 572.1563243236993), (0, 0), (4, 969.2072009703761), (0, 0), (4, 0), (34, 347.5229347303022), (4, 0), (0, 0), (38, 0), (18, 719.7434929949859), (38, 0), (0, 0), (4, 0), (37, 958.6006540282038), (4, 0), (0, 0), (46, 0), (42, 491.2162873601664), (46, 0), (0, 0), (46, 0), (48, 0), (25, 42.57255968803059), (48, 521.1517300690663), (46, 0), (0, 0), (17, 758.7610103053748), (0, 0), (8, 272.96932028331383), (0, 0), (46, 0), (7, 449.9121406416449), (46, 0), (0, 0), (24, 416.391594768439), (0, 0), (12, 203.16100143043508), (0, 0), (41, 487.17167250732217), (0, 0), (41, 0), (36, 23.7810671586678), (41, 0), (0, 0), (12, 0), (10, 902.6997941472978), (12, 0), (0, 0), (26, 0), (9, 97.2945711913458), (26, 494.49682842527443), (0, 0), (27, 0), (3, 266.60409149806003), (27, 330.53135112045254), (0, 0), (6, 717.1732989698367), (0,